This notebook is for seeing the effect msh_qual filter has on conclusions for area strains.

Previous filters for area strains:
    
* Excluding the PVs **(always)**

New:

* Remove bad quality mesh elements using the Jacobian filter AWCL introduced. Termed **msh_qual**

This notebook is for comparing:

1. area_strains_excl PVs
2. area_strains_excl_PVs_mshqual

Likewise, for fiber strains, we are comparigni:

1. percent_regional_strains/endo_avg_excl_PVs_mean

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from hrs_23_figures import *

from sklearn import metrics

regions=['global', 'roof', 'sept', 'lat', 'ant', 'post']

In [ ]:
DataPath

In [ ]:
len(nonaf_cases)

In [ ]:
len(af_cases)

In [ ]:
path="/home/csi20/Dropbox/phd/Data/kiru_demographics_df_omitAFimaging.csv"

In [ ]:
cases_df = pd.read_csv(path)

In [ ]:
## Loading Fiber Strains

## original strain calc using percentiles strategy
fiber_strains = np.zeros((cases_df['Case'].shape[0], len(regions), 2))

for i in range(0, len(regions)):
    region = regions[i]
    
    for j in range(0, cases_df['Case'].shape[0]):
        case=cases_df['Case'].iloc[j]
        
        if case in f20_cases:
            filepath=f"{DataPath}/{case}/MT-HiRes-TDownsampled/SW-0.0-BE-1e-9/percent_regional_strains"

        else:
            filepath=f'{DataPath}/{case}/MT-HiRes/SW-0.0-BE-1e-9/percent_regional_strains'
            
        data = np.loadtxt(f"{filepath}/endo_avg_excl_PVs_percent_meanstrains_{region}.txt")
        res_f1 = np.ptp(data[0, :])
        res_f2 = np.ptp(data[1, :])
        
        fiber_strains[j, i, 0] = res_f1
        fiber_strains[j, i, 1] = res_f2

In [ ]:
cases_df.head(1)

In [ ]:
for i in range(0, len(regions)):
    region = regions[i]

    cases_df[f"f2_{region}"] = fiber_strains[:, i, 1]

In [ ]:
# cases_df.to_csv("/home/csi20/Dropbox/phd/Data/kiru_demographics_df_omitAFimaging.csv", ',', index=False)

In [ ]:
## Loading area strains

area_strains = np.zeros((cases_df['Case'].shape[0], len(regions)))
area_strains_mshqual = np.zeros((cases_df['Case'].shape[0], len(regions)))

for i in range(0, len(regions)):
    region = regions[i]
    
    for j in range(0, cases_df['Case'].shape[0]):
        case=cases_df['Case'].iloc[j]
        
        if case in f20_cases:
            filepath=f"{DataPath}/{case}/MT-HiRes-TDownsampled/SW-0.0-BE-1e-9"

        else:
            filepath=f'{DataPath}/{case}/MT-HiRes/SW-0.0-BE-1e-9'
        
        data = np.loadtxt(f"{filepath}/area_meanstrains_{region}_excl_PVs.txt")
        res = np.ptp(data)
        
        area_strains[j, i] = res
        
        data = np.loadtxt(f"{filepath}/area_meanstrains_{region}_excl_PVs_mshqual.txt")
        res = np.ptp(data)
        
        area_strains_mshqual[j, i] = res

In [ ]:
for i in range(0, len(regions)):
    region = regions[i]

    cases_df[f"area_{region}"] = area_strains[:, i]

In [ ]:
for i in range(0, len(regions)):
    region = regions[i]

    cases_df[f"area_{region}_mshqual"] = area_strains_mshqual[:, i]

In [ ]:
af_df = cases_df[cases_df['af_num'] == 1]
naf_df = cases_df[cases_df['af_num'] == 0]

In [ ]:
af_df.columns

## Boxplot Figure

In [ ]:
## Combining plots: FIG 2

fig, ((ax1, ax2),
      (ax3, ax4)) = plt.subplots(2,2,figsize=(10,10), facecolor='white', gridspec_kw={'width_ratios': [1, 2]})

ax1_data = [naf_df['area_global'], af_df['area_global']]

ax2_data = [naf_df['area_roof'], af_df['area_roof'],
               naf_df['area_sept'], af_df['area_sept'],
               naf_df['area_lat'], af_df['area_lat'],
               naf_df['area_ant'], af_df['area_ant'],
               naf_df['area_post'], af_df['area_post']]

ax3_data = [naf_df['area_global_mshqual'], af_df['area_global_mshqual']]

ax4_data = [naf_df['area_roof_mshqual'], af_df['area_roof_mshqual'],
               naf_df['area_sept_mshqual'], af_df['area_sept_mshqual'],
               naf_df['area_lat_mshqual'], af_df['area_lat_mshqual'],
               naf_df['area_ant_mshqual'], af_df['area_ant_mshqual'],
               naf_df['area_post_mshqual'], af_df['area_post_mshqual']]

sns.boxplot(data=ax1_data, orient='v', ax=ax1, whis=(0, 100))
sns.boxplot(data=ax2_data, orient='v', ax=ax2, whis=(0, 100))
sns.boxplot(data=ax3_data, orient='v', ax=ax3, whis=(0, 100))
sns.boxplot(data=ax4_data, orient='v', ax=ax4, whis=(0, 100))

regions_axis=['Posterior\nwall', 'Septum', 'Lateral\nwall', 'Anterior\nwall', 'Inferior\nwall']

ax1.set_ylabel("Area Strain [%]", fontsize=15)
ax3.set_ylabel("Area Strain Msh Qual [%]", fontsize=15)

ax5.set_xticks(ticks=np.arange(0.5,1.5,1))
ax5.set_xticklabels(["Global"], fontsize=16)

ax4.set_xticks(ticks=np.arange(0.5,9.5,2))
ax4.set_xticklabels(regions_axis, fontsize=16)

for i in range(0, 10, 2):
    mybox = ax2.patches[i]
    mybox.set_facecolor('tab:blue')

for i in range(1, 11, 2):
    mybox = ax2.patches[i]
    mybox.set_facecolor('tab:orange')
    
for i in range(0, 10, 2):
    mybox = ax4.patches[i]
    mybox.set_facecolor('tab:blue')

for i in range(1, 11, 2):
    mybox = ax4.patches[i]
    mybox.set_facecolor('tab:orange')
    
legend_elements = [Patch(facecolor='tab:blue', edgecolor='black',
                         label='HF Only'),
                   Patch(facecolor='tab:orange', edgecolor='black',
                         label='HF + AF')]
ax1.legend(handles=legend_elements, loc='upper left', fontsize=14)

for i in range(0, len(fig.axes)):
    fig.axes[i].tick_params(axis='y', labelsize=12)
    
#     plt.setp(fig.axes[i].get_xticklabels(), rotation=30, horizontalalignment='center')
    fig.axes[i].tick_params(axis='x', labelsize=14)
    
    fig.axes[i].set_yticks(ticks=np.arange(0.0,160,20))




plt.tight_layout()

for i in [0, 2]:
    fig.axes[i].xaxis.set_ticks_position('none') 
    fig.axes[i].set_xticklabels([" ", " "])
    
for i in [1]:
#     fig.axes[i].xaxis.set_ticks_position('none') 
    fig.axes[i].set_xticklabels([" "]*10)

# ax1.set_yticks(ticks=np.arange(0.0,100,20))
# ax2.set_yticks(ticks=np.arange(0.0,100,20))

# ax3.set_yticks(ticks=np.arange(0.0,100,20))

# ax1.set_ylim(0.0, 80)
# ax2.set_ylim(0.0, 80)

# ax3.set_ylim(0.0, 80)
# ax4.set_ylim(0.0, 80)

# ax5.set_ylim(0.0, 80)
# ax6.set_ylim(0.0, 80)
    
# plt.savefig("/home/csi20local/Dropbox/phd/Documents/P1/figures/fig2/fig2_SAN", dpi=200, bbox_inches="tight")

In [ ]:
for i in range(0, len(regions)):
    region=regions[i]

    ans = stats.ttest_ind(naf_df[f'area_{region}'], af_df[f'area_{region}'])
    ans_mshqual = stats.ttest_ind(naf_df[f'area_{region}_mshqual'], af_df[f'area_{region}_mshqual'])

    print(f"{region}\t {ans}\n\t {ans_mshqual}")

In [ ]:
## Combining plots: FIG 2

fig, ((ax1, ax2),
      (ax3, ax4),
      (ax5, ax6)) = plt.subplots(3,2,figsize=(10,10), facecolor='white', gridspec_kw={'width_ratios': [1, 2]})

ax1_data = [naf_df['area_global'], af_df['area_global']]

ax2_data = [naf_df['area_roof'], af_df['area_roof'],
               naf_df['area_sept'], af_df['area_sept'],
               naf_df['area_lat'], af_df['area_lat'],
               naf_df['area_ant'], af_df['area_ant'],
               naf_df['area_post'], af_df['area_post']]

ax3_data = [naf_df['f1_global'], af_df['f1_global']]

ax4_data = [naf_df['f1_roof'], af_df['f1_roof'],
               naf_df['f1_sept'], af_df['f1_sept'],
               naf_df['f1_lat'], af_df['f1_lat'],
               naf_df['f1_ant'], af_df['f1_ant'],
               naf_df['f1_post'], af_df['f1_post']]

ax5_data = [naf_df['f2_global'], af_df['f2_global']]

ax6_data = [naf_df['f2_roof'], af_df['f2_roof'],
               naf_df['f2_sept'], af_df['f2_sept'],
               naf_df['f2_lat'], af_df['f2_lat'],
               naf_df['f2_ant'], af_df['f2_ant'],
               naf_df['f2_post'], af_df['f2_post']]


sns.boxplot(data=ax1_data, orient='v', ax=ax1, whis=(0, 100))
sns.boxplot(data=ax2_data, orient='v', ax=ax2, whis=(0, 100))
sns.boxplot(data=ax3_data, orient='v', ax=ax3, whis=(0, 100))
sns.boxplot(data=ax4_data, orient='v', ax=ax4, whis=(0, 100))
sns.boxplot(data=ax5_data, orient='v', ax=ax5, whis=(0, 100))
sns.boxplot(data=ax6_data, orient='v', ax=ax6, whis=(0, 100))

regions_axis=['Posterior\nwall', 'Septum', 'Lateral\nwall', 'Anterior\nwall', 'Inferior\nwall']

ax1.set_ylabel("Area Strain [%]", fontsize=15)
ax3.set_ylabel("Fibre Strain [%]", fontsize=15)
ax5.set_ylabel("Cross-Fibre Strain [%]", fontsize=15)

ax5.set_xticks(ticks=np.arange(0.5,1.5,1))
ax5.set_xticklabels(["Global"], fontsize=16)

ax6.set_xticks(ticks=np.arange(0.5,9.5,2))
ax6.set_xticklabels(regions_axis, fontsize=16)

for i in range(0, 10, 2):
    mybox = ax2.patches[i]
    mybox.set_facecolor('tab:blue')

for i in range(1, 11, 2):
    mybox = ax2.patches[i]
    mybox.set_facecolor('tab:orange')
    
for i in range(0, 10, 2):
    mybox = ax4.patches[i]
    mybox.set_facecolor('tab:blue')

for i in range(1, 11, 2):
    mybox = ax4.patches[i]
    mybox.set_facecolor('tab:orange')
    
for i in range(0, 10, 2):
    mybox = ax6.patches[i]
    mybox.set_facecolor('tab:blue')

for i in range(1, 11, 2):
    mybox = ax6.patches[i]
    mybox.set_facecolor('tab:orange')

legend_elements = [Patch(facecolor='tab:blue', edgecolor='black',
                         label='HF Only'),
                   Patch(facecolor='tab:orange', edgecolor='black',
                         label='HF + AF')]
ax1.legend(handles=legend_elements, loc='upper left', fontsize=14)

for i in range(0, len(fig.axes)):
    fig.axes[i].tick_params(axis='y', labelsize=12)
    
#     plt.setp(fig.axes[i].get_xticklabels(), rotation=30, horizontalalignment='center')
    fig.axes[i].tick_params(axis='x', labelsize=14)
    
    fig.axes[i].set_yticks(ticks=np.arange(0.0,160,20))




plt.tight_layout()

for i in [0, 2]:
    fig.axes[i].xaxis.set_ticks_position('none') 
    fig.axes[i].set_xticklabels([" ", " "])
    
for i in [1, 3]:
#     fig.axes[i].xaxis.set_ticks_position('none') 
    fig.axes[i].set_xticklabels([" "]*10)

# ax1.set_yticks(ticks=np.arange(0.0,100,20))
# ax2.set_yticks(ticks=np.arange(0.0,100,20))

# ax3.set_yticks(ticks=np.arange(0.0,100,20))

# ax1.set_ylim(0.0, 80)
# ax2.set_ylim(0.0, 80)

# ax3.set_ylim(0.0, 80)
# ax4.set_ylim(0.0, 80)

# ax5.set_ylim(0.0, 80)
# ax6.set_ylim(0.0, 80)
    
# plt.savefig("/home/csi20local/Dropbox/phd/Documents/P1/figures/fig2/fig2_SAN", dpi=200, bbox_inches="tight")

In [ ]:
for i in range(0, len(regions)):
    region=regions[i]

    ans = stats.ttest_ind(naf_df[f'area_{region}'], af_df[f'area_{region}'])
    ans_f1 = stats.ttest_ind(naf_df[f'f1_{region}'], af_df[f'f1_{region}'])
    ans_f2 = stats.ttest_ind(naf_df[f'f2_{region}'], af_df[f'f2_{region}'])


    print(f"{region}\t area: {ans}\n\t f1: {ans_f1}\n\t f2: {ans_f2}\n")

## Noramlised strain FIgure

In [ ]:
AF_area_ranges.shape

In [ ]:
AF_fibre_ranges.shape

In [ ]:
AF_area_ranges_norm = AF_area_ranges/AF_area_ranges[:, 0][:, np.newaxis]
AF_fibre_ranges_norm = AF_fibre_ranges/AF_fibre_ranges[:, 0, :][:, np.newaxis]
# AF_fibre_ranges_norm = AF_fibre_ranges[:, :, 1]/AF_fibre_ranges[:, 0, 1][:, np.newaxis]

## AUC Figure 

### Todo: add in LAEF and LA volume for extra EBR cases

In [ ]:
AF_fibre_ranges.shape

In [ ]:
nonAF_fibre_ranges.shape

In [ ]:
total_cases = len(af_cases)+len(nonaf_cases)

y_true = np.zeros((total_cases, 6))
y_true[:len(nonaf_cases), :] = 1.0
y_true[len(nonaf_cases):, :] = 0.0

y_probs_fibres = np.zeros((total_cases, 6, 2))
y_probs_fibres[:len(nonaf_cases), :, 0] = nonAF_fibre_ranges[:, :, 0] 
y_probs_fibres[len(nonaf_cases):, :, 0] = AF_fibre_ranges[:, :, 0]
y_probs_fibres[:len(nonaf_cases), :, 1] = nonAF_fibre_ranges[:, :, 1] 
y_probs_fibres[len(nonaf_cases):, :, 1] = AF_fibre_ranges[:, :, 1]

y_probs_area = np.zeros((total_cases, 6))
y_probs_area[:len(nonaf_cases), :] = nonAF_area_ranges[:, :] 
y_probs_area[len(nonaf_cases):, :] = AF_area_ranges[:, :] 

# y_probs_LAEF = np.zeros((total_cases,))
# y_probs_LAEF[:len(nonaf_cases)] = nonAF_LAEF[:] 
# y_probs_LAEF[len(nonaf_cases):] = AF_LAEF[:] 

# y_probs_LAvol = np.zeros((total_cases,))
# y_probs_LAvol[:len(nonaf_cases)] = nonAF_LAvol[:] 
# y_probs_LAvol[len(nonaf_cases):] = AF_LAvol[:] 

fpr_f1, tpr_f1, thresholds_f1 = metrics.roc_curve(y_true[:, 0], y_probs_fibres[:, 0, 0])
auc_f1 = metrics.roc_auc_score(y_true[:, 0], y_probs_fibres[:, 0, 0])

fpr_f2, tpr_f2, thresholds_f2 = metrics.roc_curve(y_true[:, 0], y_probs_fibres[:, 0, 1])
auc_f2 = metrics.roc_auc_score(y_true[:, 0], y_probs_fibres[:, 0, 1])

fpr_a, tpr_a, thresholds_a = metrics.roc_curve(y_true[:, 0], y_probs_area[:, 0])
auc_a = metrics.roc_auc_score(y_true[:, 0], y_probs_area[:, 0])

# fpr_LAEF, tpr_LAEF, thresholds_LAEF = metrics.roc_curve(y_true[:, 0], y_probs_LAEF[:])
# auc_LAEF = metrics.roc_auc_score(y_true[:, 0], y_probs_LAEF[:])

# fpr_LAvol, tpr_LAvol, thresholds_LAvol = metrics.roc_curve(y_true[:, 0], y_probs_LAvol[:])
# auc_LAvol = metrics.roc_auc_score(y_true[:, 0], y_probs_LAvol[:])

print("AUC fibres f1: ", auc_f1)
print("AUC fibres f2: ", auc_f2)

print("AUC area: ", auc_a)
print("AUC LAEF: ", auc_LAEF)
print("AUC LAvol: ", auc_LAvol)

In [ ]:
## Plotting all 6 lines on same plot

fig, ((ax1, ax2, ax3, ax4)) = plt.subplots(1,4,figsize=(24,5), facecolor='white', sharey=True)

regions_axis=['Global', 'Roof', 'Septum', 'Lateral wall', 'Anterior wall', 'Posterior wall']

for i in range(0, len(regions)):
    
    fpr, tpr, thresholds = metrics.roc_curve(y_true[:, i], y_probs_area[:, i])
    auc = metrics.roc_auc_score(y_true[:, i], y_probs_area[:, i])
    
    ax1.plot(fpr, tpr, label=f"{regions_axis[i]}, AUC = {str(np.round(auc,2))}", lw='2')

for i in range(0, len(regions)):
    
    fpr, tpr, thresholds = metrics.roc_curve(y_true[:, i], y_probs_fibres[:, i, 0])
    auc = metrics.roc_auc_score(y_true[:, i], y_probs_fibres[:, i, 0])
    
    ax2.plot(fpr, tpr, label=f"{regions_axis[i]}, AUC = {str(np.round(auc,2))}", lw='2')
    
for i in range(0, len(regions)):
    
    fpr, tpr, thresholds = metrics.roc_curve(y_true[:, i], y_probs_fibres[:, i, 1])
    auc = metrics.roc_auc_score(y_true[:, i], y_probs_fibres[:, i, 1])
    
    ax3.plot(fpr, tpr, label=f"{regions_axis[i]}, AUC = {str(np.round(auc,2))}", lw='2')

fpr_LAEF, tpr_LAEF, thresholds_LAEF = metrics.roc_curve(y_true[:, 0], y_probs_LAEF[:])
auc_LAEF = metrics.roc_auc_score(y_true[:, 0], y_probs_LAEF[:])

# fpr_LAvol, tpr_LAvol, thresholds_LAvol = metrics.roc_curve(y_true[:, 0], -y_probs_LAvol[:])
# auc_LAvol = metrics.roc_auc_score(y_true[:, 0], -y_probs_LAvol[:])

# ax4.plot(fpr_LAEF, tpr_LAEF, label=f"LAEF, AUC = {str(np.round(auc_LAEF,2))}", lw='2')
# ax4.plot(fpr_LAvol, tpr_LAvol, label=f"LA vol, AUC = {str(np.round(auc_LAvol,2))}", lw='2')

for i in range(0, len(fig.axes)):
    fig.axes[i].plot(np.arange(0,1.1,0.1), np.arange(0,1.1,0.1), ls='--', c='black')
    fig.axes[i].set_xlabel("False Positive Rate", fontsize=15)
    fig.axes[i].set_ylabel("True Positive Rate", fontsize=15)
    fig.axes[i].legend(fontsize=14)
    
plt.tight_layout()

ax1.set_title("Area Strain", fontsize=16)
ax2.set_title("Fiber Strain", fontsize=16)
ax3.set_title("Cross-Fiber Strain", fontsize=16)
ax4.set_title("LAEF & LA volume", fontsize=16)

# plt.text(0.6, 0.5, f'AUC = {str(np.round(auc,3))}', size=15)
# plt.savefig("/home/csi20/Dropbox/phd/Documents/P1/figures/fig3/fig3_2.png", dpi=200, bbox_inches="tight")

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

kf = KFold(n_splits=5, shuffle=False)
kf_strat = StratifiedKFold(n_splits=5, shuffle=False)

## Want to use straitifed kfold as want roughly same proportion of AF and nonAF in each fold

## use indices for train and test splits to create pred and true arrays for each fold
## calculate auc for each fold
## calculate roc values for each fold
## use this to calculate mean, std of AUC
## Use this to calculate mean ROC curve coordiantes

In [ ]:
all_cases = nonaf_cases + af_cases

In [ ]:
af_naf_classes = np.zeros((len(all_cases),))
af_naf_classes[:len(nonaf_cases)] = 0.0
af_naf_classes[len(nonaf_cases):] = 1.0

In [ ]:
kf_strat_split = kf_strat.split(all_cases, af_naf_classes)

In [ ]:
kf_strat_split_inst = next(kf_strat_split)

training_list = [all_cases[i] for i in kf_strat_split_inst[0]]
val_list = [all_cases[i] for i in kf_strat_split_inst[1]]

In [ ]:
all_area_ranges = np.concatenate((nonAF_area_ranges, AF_area_ranges))
all_fibre_ranges = np.concatenate((nonAF_fibre_ranges, AF_fibre_ranges))

In [ ]:
total_cases

In [ ]:
from sklearn.metrics import auc


## Populate y_true and y_pred labels using ALL data
y_true = np.zeros((total_cases, 6))
y_true[:len(nonaf_cases), :] = 1.0
y_true[len(nonaf_cases):, :] = 0.0

y_probs_fibres = np.zeros((total_cases, 6, 2))
y_probs_fibres[:len(nonaf_cases), :, 0] = nonAF_fibre_ranges[:, :, 0] 
y_probs_fibres[len(nonaf_cases):, :, 0] = AF_fibre_ranges[:, :, 0]
y_probs_fibres[:len(nonaf_cases), :, 1] = nonAF_fibre_ranges[:, :, 1] 
y_probs_fibres[len(nonaf_cases):, :, 1] = AF_fibre_ranges[:, :, 1]

y_probs_area = np.zeros((total_cases, 6))
y_probs_area[:len(nonaf_cases), :] = nonAF_area_ranges[:, :] 
y_probs_area[len(nonaf_cases):, :] = AF_area_ranges[:, :] 

# y_probs_LAEF = np.zeros((total_cases,))
# y_probs_LAEF[:len(nonaf_cases)] = nonAF_LAEF[:] 
# y_probs_LAEF[len(nonaf_cases):] = AF_LAEF[:] 

# y_probs_LAvol = np.zeros((total_cases,))
# y_probs_LAvol[:len(nonaf_cases)] = nonAF_LAvol[:] 
# y_probs_LAvol[len(nonaf_cases):] = AF_LAvol[:] 

## Initialise results arrays

## Shape: K_folds X regions
area_auc = np.zeros((5,6))
fibre_f1_auc = np.zeros((5,6))
fibre_f2_auc = np.zeros((5,6))
LAEF_auc = np.zeros((5,))
LAvol_auc = np.zeros((5,))

## Interpolate TPRs 
## Fold X Interpolated size X Region
tprs_f1 = np.zeros((5, 100, 6))
tprs_f2 = np.zeros((5, 100, 6))
tprs_a = np.zeros((5, 100, 6))

## Fold X Interpolated size
tprs_LAEF = np.zeros((5, 100))
tprs_LAvol = np.zeros((5, 100))

mean_fpr = np.linspace(0, 1, 100)

for i, (train_index, test_index) in enumerate(kf_strat.split(all_cases, af_naf_classes)):
    print(f"Fold {i}:")
    
    ## Create pred and true arrays for fold
    y_true_fold = np.take(y_true, train_index, axis=0)
    
    y_probs_fibres_fold = np.take(y_probs_fibres, train_index, axis=0) 
    y_probs_area_fold = np.take(y_probs_area, train_index, axis=0)
#     y_probs_LAEF_fold = np.take(y_probs_LAEF, train_index, axis=0)
#     y_probs_LAvol_fold = np.take(y_probs_LAvol, train_index, axis=0)
    
    ## Calculate ROC scores
    
    ## iterate over all regions
    for j in range(0, 6):
        
        ## Fiber strains
        fpr_f1, tpr_f1, thresholds_f1 = metrics.roc_curve(y_true_fold[:, j], y_probs_fibres_fold[:, j, 0])
        auc_f1 = metrics.roc_auc_score(y_true_fold[:, j], y_probs_fibres_fold[:, j, 0])
        fibre_f1_auc[i, j] = auc_f1
        
        ## Interpolate tpr and fpr values
        interp_tpr = np.interp(mean_fpr, fpr_f1, tpr_f1)
        interp_tpr[0] = 0.0
        tprs_f1[i, :, j] = interp_tpr

        ## Cross-fiber strains
        fpr_f2, tpr_f2, thresholds_f2 = metrics.roc_curve(y_true_fold[:, j], y_probs_fibres_fold[:, j, 1])
        auc_f2 = metrics.roc_auc_score(y_true_fold[:, j], y_probs_fibres_fold[:, j, 1])
        fibre_f2_auc[i, j] = auc_f2
        
        ## Interpolate tpr and fpr values
        interp_tpr = np.interp(mean_fpr, fpr_f2, tpr_f2)
        interp_tpr[0] = 0.0
        tprs_f2[i, :, j] = interp_tpr
        
        ## Area strains
        fpr_a, tpr_a, thresholds_a = metrics.roc_curve(y_true_fold[:, j], y_probs_area_fold[:, j])
        auc_a = metrics.roc_auc_score(y_true_fold[:, j], y_probs_area_fold[:, j])
        area_auc[i, j] = auc_a
        
        ## Interpolate tpr and fpr values
        interp_tpr = np.interp(mean_fpr, fpr_a, tpr_a)
        interp_tpr[0] = 0.0
        tprs_a[i, :, j] = interp_tpr

    ## LAEF
    fpr_LAEF, tpr_LAEF, thresholds_LAEF = metrics.roc_curve(y_true_fold[:, 0], y_probs_LAEF_fold[:])
    auc_LAEF = metrics.roc_auc_score(y_true_fold[:, 0], y_probs_LAEF_fold[:])
    LAEF_auc[i] = auc_LAEF
    
    ## Interpolate tpr and fpr values
    interp_tpr = np.interp(mean_fpr, fpr_LAEF, tpr_LAEF)
    interp_tpr[0] = 0.0
    tprs_LAEF[i, :] = interp_tpr

#     ## LA volume
#     fpr_LAvol, tpr_LAvol, thresholds_LAvol = metrics.roc_curve(y_true_fold[:, 0], -y_probs_LAvol_fold[:])
#     auc_LAvol = metrics.roc_auc_score(y_true_fold[:, 0], -y_probs_LAvol_fold[:])
#     LAvol_auc[i] = auc_LAvol
    
#     ## Interpolate tpr and fpr values
#     interp_tpr = np.interp(mean_fpr, fpr_LAvol, tpr_LAvol)
#     interp_tpr[0] = 0.0
#     tprs_LAvol[i, :] = interp_tpr

## Calculate mean tpr over folds

mean_tpr_f1 = np.mean(tprs_f1, axis=0)
mean_tpr_f2 = np.mean(tprs_f2, axis=0)
mean_tpr_a = np.mean(tprs_a, axis=0)
mean_tpr_LAEF = np.mean(tprs_LAEF, axis=0)
mean_tpr_LAvol = np.mean(tprs_LAvol, axis=0)

## Set final coordinate as 1

mean_tpr_f1[-1, :] = 1.0
mean_tpr_f2[-1, :] = 1.0
mean_tpr_a[-1, :] = 1.0
mean_tpr_LAEF[-1] = 1.0
mean_tpr_LAvol[-1] = 1.0

mean_auc_f1 = np.mean(fibre_f1_auc, axis=0)
mean_auc_f2 = np.mean(fibre_f2_auc, axis=0)
mean_auc_a = np.mean(area_auc, axis=0)
mean_auc_LAEF = np.mean(LAEF_auc, axis=0)
mean_auc_LAvol = np.mean(LAvol_auc, axis=0)

std_auc_f1 = np.std(fibre_f1_auc, axis=0)
std_auc_f2 = np.std(fibre_f2_auc, axis=0)
std_auc_a = np.std(area_auc, axis=0)
std_auc_LAEF = np.std(LAEF_auc, axis=0)
std_auc_LAvol = np.std(LAvol_auc, axis=0)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(12,12), facecolor='white', sharey=True)

regions_axis=['Global', 'Posterior wall', 'Septum', 'Lateral wall', 'Anterior wall', 'Inferior wall']

ax1.plot(mean_fpr, mean_tpr_a[:, 0], 
         label=f"{regions_axis[0]}, AUC: {np.round(mean_auc_a[0], 2)} $\pm$ {np.round(std_auc_a[0], 2)}", lw=2, 
         color='black')
ax2.plot(mean_fpr, mean_tpr_f1[:, 0], 
         label=f"{regions_axis[0]}, AUC: {np.round(mean_auc_f1[0], 2)} $\pm$ {np.round(std_auc_f1[0], 2)}", lw=2, 
         color='black')    
ax3.plot(mean_fpr, mean_tpr_f2[:, 0], 
         label=f"{regions_axis[0]}, AUC: {np.round(mean_auc_f2[0], 2)} $\pm$ {np.round(std_auc_f2[0], 2)}", lw=2, 
         color='black')    

for i in range(1, len(regions)):
    ax2.plot(mean_fpr, mean_tpr_f1[:, i], 
             label=f"{regions_axis[i]}, AUC: {np.round(mean_auc_f1[i], 2)} $\pm$ {np.round(std_auc_f1[i], 2)}", lw=2)
    
    ax3.plot(mean_fpr, mean_tpr_f2[:, i], 
             label=f"{regions_axis[i]}, AUC: {np.round(mean_auc_f2[i], 2)} $\pm$ {np.round(std_auc_f2[i], 2)}", lw=2)
    
    ax1.plot(mean_fpr, mean_tpr_a[:, i], 
             label=f"{regions_axis[i]}, AUC: {np.round(mean_auc_a[i], 2)} $\pm$ {np.round(std_auc_a[i], 2)}", lw=2)
    
ax4.plot(mean_fpr, mean_tpr_LAEF, 
             label=f"LAEF, AUC: {np.round(mean_auc_LAEF, 2)} $\pm$ {np.round(std_auc_LAEF, 2)}", lw=2)
    
ax4.plot(mean_fpr, mean_tpr_LAvol, 
             label=f"LA EDV, AUC: {np.round(mean_auc_LAvol, 2)} $\pm$ {np.round(std_auc_LAvol, 2)}", lw=2)

for i in range(0, len(fig.axes[:])):
#     fig.axes[i].legend(fontsize=14, bbox_to_anchor=(0.3, 0.55))
    fig.axes[i].plot(np.arange(0,1.1,0.1), np.arange(0,1.1,0.1), ls='--', c='black')
    fig.axes[i].tick_params(axis='both', labelsize=13)

fig.axes[0].set_ylabel("True Positive Rate", fontsize=15)
fig.axes[2].set_ylabel("True Positive Rate", fontsize=15)
fig.axes[2].set_xlabel("False Positive Rate", fontsize=15)
fig.axes[3].set_xlabel("False Positive Rate", fontsize=15)

for i in range(0, 4, 1):
    fig.axes[i].legend(fontsize=14, loc='lower right')

ax1.set_title("Area Strain", fontsize=16)
ax2.set_title("Fiber Strain", fontsize=16)
ax3.set_title("Cross-Fiber Strain", fontsize=16)
ax4.set_title("LAEF & LA EDV", fontsize=16)
plt.tight_layout()

# plt.savefig("/home/csi20local/Dropbox/phd/Documents/P1/roc_curves_cval_SAN_blackglobal.png", dpi=200, bbox_inches="tight")

In [ ]:
y_probs_fibres.shape

In [ ]:
## Investigating if adding strain to LAEF and LAEDV imrpoves AUC

Y_pred_area_norm = (y_probs_fibres[:, 0, 1] - y_probs_fibres[:, 0, 1].mean())/y_probs_fibres[:, 0, -0].std()
Y_pred_LAEF_norm = (y_probs_LAvol - y_probs_LAvol.mean())/y_probs_LAvol.std()

Y_pred_overall = Y_pred_area_norm - Y_pred_LAEF_norm

ans = metrics.roc_auc_score(y_true[:, 0], -Y_pred_overall)
1-ans